#### Initial setup

In [4]:
# Project info
PROJECT_ID = 'marvin-voice'
GIT_OWNER, GIT_REPO = 'patrikkj', 'marvin-models'
DATASET = 'dataset'

# Set current working directory to project root
import os
if os.getcwd().endswith("notebooks"):
    %cd -q ..
    
# Identify environment
IN_COLAB = 'google.colab' in str(get_ipython())

In [4]:
# Run this cell only if we're running from Google Colab
if IN_COLAB:
    !git clone -q https://github.com/{GIT_OWNER}/{GIT_REPO}.git
    !gsutil -q cp gs://{PROJECT_ID}/data/raw/{DATASET}.tar.gz /tmp
    !tar -xf /tmp/{DATASET}.tar.gz -C /content/{GIT_REPO}/data
    import sys
    sys.path.append(f"/content/{GIT_REPO}")
else:
    !gsutil -q cp gs://{PROJECT_ID}/data/raw/{DATASET}.tar.gz data
    !tar -xf data/{DATASET}.tar.gz -C data

In [5]:
# Internal modules
import os, sys, glob
from importlib import reload

# External modules
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_io as tfio
from tensorflow_io import experimental as tfex

# Colab modules
if IN_COLAB:
    from google.colab import auth
    from IPython import display
    from oauth2client.client import GoogleCredentials
    from pydrive.auth import GoogleAuth
    
# Scripts
from scripts import preprocessing

In [119]:
preprocessing = reload(preprocessing)

paths = glob.glob(f"data/dataset/*/*.wav")
#paths = paths[:500] + paths[-500:]

tensors = [preprocessing.to_tensor(path) for path in paths]
specs = [preprocessing.tensor_to_log_mel_spec(tensor) for tensor in tensors]
mfccs = [preprocessing.batch_log_mel_spec_to_mfccs(spec) for spec in specs]

labels = [preprocessing.to_binary_label(path) for path in paths]
#dataset = tf.data.Dataset.from_tensor_slices((specs, labels))


In [129]:
dataset = tf.data.Dataset.from_tensor_slices((mfccs, labels))
"""
for tensor, label in dataset.take(5):
    print(tensor.shape)
    print(label.shape)
    print()
"""
dataset = dataset.shuffle(10000).batch(512)

model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='sigmoid'),
  tf.keras.layers.Dense(16, activation='sigmoid'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(dataset, epochs=50)

Epoch 1/50
2/2 [==============================] - 0s 3ms/step - loss: 0.6855 - accuracy: 0.5300
Epoch 2/50
2/2 [==============================] - 0s 3ms/step - loss: 0.6441 - accuracy: 0.7790
Epoch 3/50
2/2 [==============================] - 0s 3ms/step - loss: 0.6205 - accuracy: 0.8380
Epoch 4/50
2/2 [==============================] - 0s 2ms/step - loss: 0.5923 - accuracy: 0.8420
Epoch 5/50
2/2 [==============================] - 0s 3ms/step - loss: 0.5572 - accuracy: 0.8720
Epoch 6/50
2/2 [==============================] - 0s 3ms/step - loss: 0.5323 - accuracy: 0.8870
Epoch 7/50
2/2 [==============================] - 0s 2ms/step - loss: 0.5100 - accuracy: 0.8920
Epoch 8/50
2/2 [==============================] - 0s 3ms/step - loss: 0.4758 - accuracy: 0.8980
Epoch 9/50
2/2 [==============================] - 0s 3ms/step - loss: 0.4437 - accuracy: 0.8940
Epoch 10/50
2/2 [==============================] - 0s 3ms/step - loss: 0.4212 - accuracy: 0.8940
Epoch 11/50
2/2 [======================

In [8]:
from scripts import record_audio
audio, sr = record_audio.get_audio()

ModuleNotFoundError: No module named 'google.colab'